In [13]:
import laspy as lp
import numpy as np

In [34]:
import os
import laspy as lp
import numpy as np
import pylas
folder = "sample_roofdata_50"
for roof in os.listdir(folder):
    if roof.endswith(".laz"):
        # Lag hele filstien ved å kombinere mappen og filnavnet
        file_path = os.path.join(folder, roof)
        
        # Åpne filen med laspy
        with lp.open(file_path) as fh:
            print(f"Loaded file: {roof}")
            print(f"Number of points: {fh.header.point_count}")



Loaded file: 182130478.laz
Number of points: 1431
Loaded file: 10566010.laz
Number of points: 610
Loaded file: 182250287.laz
Number of points: 2762
Loaded file: 182213209.laz
Number of points: 3732
Loaded file: 182461407.laz
Number of points: 1463
Loaded file: 182380105.laz
Number of points: 2242
Loaded file: 182251615.laz
Number of points: 3642
Loaded file: 182221864.laz
Number of points: 3215
Loaded file: 182486507.laz
Number of points: 7653
Loaded file: 182251399.laz
Number of points: 4449
Loaded file: 10565839.laz
Number of points: 514
Loaded file: 182307580.laz
Number of points: 1702
Loaded file: 182294063.laz
Number of points: 1494
Loaded file: 10529360.laz
Number of points: 2571
Loaded file: 182292761.laz
Number of points: 1134
Loaded file: 10479436.laz
Number of points: 1571
Loaded file: 182311324.laz
Number of points: 4134
Loaded file: 182217581.laz
Number of points: 2518
Loaded file: 182461857.laz
Number of points: 3411
Loaded file: 182379476.laz
Number of points: 2605
Loaded